In [1]:
# Scientific libraries
              
import numpy as np              
import scipy as sp              
import pandas as pd              
import json              
import missingno as msno              

# Loading Plotting Modules
              
import matplotlib              
import matplotlib.pyplot as plt              
import seaborn as sns              
%matplotlib inline              
import chart_studio.plotly as py              
import plotly.figure_factory as ff              
import plotly.graph_objects as go              
import plotly.express as px              
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot              
init_notebook_mode(connected=True)              

# Setting Data Frame Options
              
pd.set_option('display.max_rows', 40)              
pd.set_option('display.max_columns', 50)              
pd.set_option('display.width', 100)              
pd.set_option('display.max_colwidth', 100)              

# Setting Plot Configuration
              
sns.set(rc={'figure.figsize':(19,11)}, style = 'white')              

import katana.nbutil as ktn
              
import os

In [2]:
df = pd.read_parquet('./output/pre-modeling/reduced_all_cat.pqt.gz')                
m, n = df.shape
print('Data Frame: {} x {}'.format(m,n))                
df.head()

Data Frame: 1097231 x 6202


,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W,card2_100.0,card2_101.0,card2_102.0,card2_103.0,card2_104.0,card2_105.0,card2_106.0,card2_107.0,card2_108.0,card2_109.0,card2_110.0,card2_111.0,card2_112.0,card2_113.0,card2_114.0,card2_115.0,card2_116.0,card2_117.0,card2_118.0,card2_119.0,...,DeviceInfo_rv:49.0,DeviceInfo_rv:50.0,DeviceInfo_rv:51.0,DeviceInfo_rv:52.0,DeviceInfo_rv:52.9,DeviceInfo_rv:53.0,DeviceInfo_rv:54.0,DeviceInfo_rv:55.0,DeviceInfo_rv:56.0,DeviceInfo_rv:57.0,DeviceInfo_rv:58.0,DeviceInfo_rv:59.0,DeviceInfo_rv:60.0,DeviceInfo_rv:61.0,DeviceInfo_rv:62.0,DeviceInfo_rv:63.0,DeviceInfo_rv:64.0,DeviceInfo_rv:65.0,DeviceInfo_verykoolS5005,DeviceInfo_verykoolS5525,DeviceInfo_verykoolS5530 Build/LMY47I,DeviceInfo_verykools4009,DeviceInfo_verykools5004,DeviceInfo_vivo,DeviceInfo_nan
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2987001,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2987002,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2987003,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2987004,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
y = pd.read_parquet('./output/pre-modeling/all_num_data.pqt.gz', columns=['isFraud'])

In [4]:
df['isFraud'] = y.isFraud

In [5]:
import gc
del y
gc.collect()

19829

In [43]:
a = pd.DataFrame(np.random.randint(0,2,(100,6)), columns = list('ABCDEF'))

In [45]:
tabs = [pd.crosstab(a.A,a[col]) for col in a.drop('A',axis=1).columns]

In [58]:
tabs[0]

B,0,1
A,,
0,29,23
1,31,17


In [59]:
29*17 - 23*31

-220

In [155]:
def random_fixed_sum(low=0,high=None,size=(2,2), sum_=100):
    if not high:
        high = sum_+1
    arr = np.random.randint(low, high, size)
    if arr.sum() != sum_ or any(arr.ravel()==0):
        return random_fixed_sum(low,high,size,sum_)
    return arr

In [64]:
np.array(trial).sum()

100

In [156]:
def odds(arr: list):
    if len(arr)!=2:
        return
    return arr[0]/arr[1]

In [327]:
def odds_ratio(mat, log=False, positive_only=False):
    if positive_only:
        if isinstance(mat, pd.DataFrame):
            return mat.loc[1,1]/mat.loc[0,1]
        return mat[1][1]/mat[0][1]
    odds1 = odds(mat[1])
    odds2 = odds(mat[0])
    
    if log:
        return np.log(odds1/odds2)
    return odds1/odds2


In [228]:
def yules_q(x, absolute=False):
    if isinstance(x,float):
        return abs((x-1)/(x+1)) if absolute else (x-1)/(x+1)
    else:
        return yules_q(odds_ratio(x, positive_only=True), absolute=True) if absolute\
                else yules_q(odds_ratio(x, positive_only=True))

In [221]:
def concordance_report(mat):
    print('*'*58)
    print(mat)
    print('Odds Ratio: %.3f' % odds_ratio(mat))
    print('Odds Ratio (pos): %.3f' % odds_ratio(mat, positive_only=True))
    print('Log Odds Ratio: %.3f' % odds_ratio(mat, log=True))
    print("Yule's Q (abs): %.3f" % yules_q(mat, absolute=True))
    print('*'*58)

In [109]:
odds_ratio(trial, log=True)

1.9363406980391626

In [312]:
trial = random_fixed_sum()
# trial = np.array([[82,5],[10,3]])
# np.linalg.norm(trial, ord=1)
concordance_report(trial)

**********************************************************
[[37 11]
 [24 28]]
Odds Ratio: 0.255
Odds Ratio (pos): 2.545
Log Odds Ratio: -1.367
Yule's Q (abs): 0.436
**********************************************************


In [328]:
tabs = []
for col in df.columns:
    if col == 'isFraud':
        continue
    tab = pd.crosstab(df.isFraud,df[col])
    try:
        sum_1 = tab.loc[:,tab.columns.get_level_values(0)==1].sum().values[0]
    except Exception as e:
        print('Exception occurred during summation. Message: %s' % e)
        sum_1 = 0
    
    if tab.shape != (2,2):
        tabs.append([col, -1, sum_1])
        continue
    concordance_report(tab)
    tabs.append([col, odds_ratio(tab, positive_only=True), sum_1])

**********************************************************
ProductCD_C       0      1
isFraud                   
0.0          509366  60511
1.0           12655   8008
Odds Ratio: 0.188
Odds Ratio (pos): 0.132
Log Odds Ratio: -1.673
Yule's Q (abs): 0.766
**********************************************************
**********************************************************
ProductCD_H       0      1
isFraud                   
0.0          538427  31450
1.0           19089   1574
Odds Ratio: 0.708
Odds Ratio (pos): 0.050
Log Odds Ratio: -0.345
Yule's Q (abs): 0.905
**********************************************************
**********************************************************
ProductCD_R       0      1
isFraud                   
0.0          533604  36273
1.0           19237   1426
Odds Ratio: 0.917
Odds Ratio (pos): 0.039
Log Odds Ratio: -0.087
Yule's Q (abs): 0.924
**********************************************************
**********************************************************
Pr

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning:

divide by zero encountered in long_scalars



**********************************************************
card2_104.0       0    1
isFraud                 
0.0          569310  567
1.0           20476  187
Odds Ratio: 0.109
Odds Ratio (pos): 0.330
Log Odds Ratio: -2.216
Yule's Q (abs): 0.504
**********************************************************
**********************************************************
card2_105.0       0    1
isFraud                 
0.0          569587  290
1.0           20663    0
Odds Ratio: inf
Odds Ratio (pos): 0.000
Log Odds Ratio: inf
Yule's Q (abs): 1.000
**********************************************************
**********************************************************
card2_106.0       0    1
isFraud                 
0.0          569615  262
1.0           20663    0
Odds Ratio: inf
Odds Ratio (pos): 0.000
Log Odds Ratio: inf
Yule's Q (abs): 1.000
**********************************************************
Exception occurred during summation. Message: index 0 is out of bounds for axis 0 with size 0
*

**********************************************************
card2_134.0       0    1
isFraud                 
0.0          569407  470
1.0           20643   20
Odds Ratio: 0.852
Odds Ratio (pos): 0.043
Log Odds Ratio: -0.160
Yule's Q (abs): 0.918
**********************************************************
**********************************************************
card2_135.0       0   1
isFraud                
0.0          569809  68
1.0           20663   0
Odds Ratio: inf
Odds Ratio (pos): 0.000
Log Odds Ratio: inf
Yule's Q (abs): 1.000
**********************************************************
**********************************************************
card2_136.0       0   1
isFraud                
0.0          569809  68
1.0           20663   0
Odds Ratio: inf
Odds Ratio (pos): 0.000
Log Odds Ratio: inf
Yule's Q (abs): 1.000
**********************************************************
**********************************************************
card2_137.0       0   1
isFraud             

**********************************************************
card2_163.0       0   1
isFraud                
0.0          569817  60
1.0           20663   0
Odds Ratio: inf
Odds Ratio (pos): 0.000
Log Odds Ratio: inf
Yule's Q (abs): 1.000
**********************************************************
**********************************************************
card2_164.0       0    1
isFraud                 
0.0          569766  111
1.0           20661    2
Odds Ratio: 2.013
Odds Ratio (pos): 0.018
Log Odds Ratio: 0.699
Yule's Q (abs): 0.965
**********************************************************
**********************************************************
card2_165.0       0    1
isFraud                 
0.0          569762  115
1.0           20662    1
Odds Ratio: 4.170
Odds Ratio (pos): 0.009
Log Odds Ratio: 1.428
Yule's Q (abs): 0.983
**********************************************************
**********************************************************
card2_166.0       0   1
isFraud      

...

**********************************************************
DeviceInfo_Z831       0  1
isFraud                   
0.0              569876  1
1.0               20663  0
Odds Ratio: inf
Odds Ratio (pos): 0.000
Log Odds Ratio: inf
Yule's Q (abs): 1.000
**********************************************************
**********************************************************
DeviceInfo_Z832 Build/MMB29M       0  1
isFraud                                
0.0                           569876  1
1.0                            20658  5
Odds Ratio: 0.007
Odds Ratio (pos): 5.000
Log Odds Ratio: -4.927
Yule's Q (abs): 0.667
**********************************************************
**********************************************************
DeviceInfo_Z833       0  1
isFraud                   
0.0              569875  2
1.0               20663  0
Odds Ratio: inf
Odds Ratio (pos): 0.000
Log Odds Ratio: inf
Yule's Q (abs): 1.000
**********************************************************
*******************

**********************************************************
DeviceInfo_ZTE BLADE V8 MINI Build/NRD90M       0  1
isFraud                                             
0.0                                        569872  5
1.0                                         20663  0
Odds Ratio: inf
Odds Ratio (pos): 0.000
Log Odds Ratio: inf
Yule's Q (abs): 1.000
**********************************************************
**********************************************************
DeviceInfo_ZTE Blade A511 Build/MRA58K       0  1
isFraud                                          
0.0                                     569876  1
1.0                                      20658  5
Odds Ratio: 0.007
Odds Ratio (pos): 5.000
Log Odds Ratio: -4.927
Yule's Q (abs): 0.667
**********************************************************
**********************************************************
DeviceInfo_ZTE Blade L5 Build/LMY47I       0   1
isFraud                                         
0.0                       

Exception occurred during summation. Message: index 0 is out of bounds for axis 0 with size 0
Exception occurred during summation. Message: index 0 is out of bounds for axis 0 with size 0
Exception occurred during summation. Message: index 0 is out of bounds for axis 0 with size 0
Exception occurred during summation. Message: index 0 is out of bounds for axis 0 with size 0
Exception occurred during summation. Message: index 0 is out of bounds for axis 0 with size 0
Exception occurred during summation. Message: index 0 is out of bounds for axis 0 with size 0
Exception occurred during summation. Message: index 0 is out of bounds for axis 0 with size 0
Exception occurred during summation. Message: index 0 is out of bounds for axis 0 with size 0
Exception occurred during summation. Message: index 0 is out of bounds for axis 0 with size 0
Exception occurred during summation. Message: index 0 is out of bounds for axis 0 with size 0
Exception occurred during summation. Message: index 0 is out

**********************************************************
DeviceInfo_rv:43.0       0  1
isFraud                      
0.0                 569871  6
1.0                  20660  3
Odds Ratio: 0.073
Odds Ratio (pos): 0.500
Log Odds Ratio: -2.624
Yule's Q (abs): 0.333
**********************************************************
**********************************************************
DeviceInfo_rv:44.0       0  1
isFraud                      
0.0                 569872  5
1.0                  20663  0
Odds Ratio: inf
Odds Ratio (pos): 0.000
Log Odds Ratio: inf
Yule's Q (abs): 1.000
**********************************************************
**********************************************************
DeviceInfo_rv:45.0       0   1
isFraud                       
0.0                 569846  31
1.0                  20662   1
Odds Ratio: 1.124
Odds Ratio (pos): 0.032
Log Odds Ratio: 0.117
Yule's Q (abs): 0.938
**********************************************************
***************************

**********************************************************
DeviceInfo_verykools5004       0  1
isFraud                            
0.0                       569875  2
1.0                        20663  0
Odds Ratio: inf
Odds Ratio (pos): 0.000
Log Odds Ratio: inf
Yule's Q (abs): 1.000
**********************************************************
**********************************************************
DeviceInfo_vivo       0  1
isFraud                   
0.0              569872  5
1.0               20663  0
Odds Ratio: inf
Odds Ratio (pos): 0.000
Log Odds Ratio: inf
Yule's Q (abs): 1.000
**********************************************************
**********************************************************
DeviceInfo_nan       0       1
isFraud                       
0.0             110059  459818
1.0               8607   12056
Odds Ratio: 2.983
Odds Ratio (pos): 0.026
Log Odds Ratio: 1.093
Yule's Q (abs): 0.949
**********************************************************


In [329]:
tabs

[['ProductCD_C', 0.13233957462279586, 68519],
 ['ProductCD_H', 0.05004769475357711, 33024],
 ['ProductCD_R', 0.03931298762164696, 37699],
 ['ProductCD_S', 0.06269420581246572, 11628],
 ['ProductCD_W', 0.020824191260294264, 439670],
 ['card2_100.0', 0.012980061554931085, 7570],
 ['card2_101.0', 0.0, 156],
 ['card2_102.0', 0.0, 332],
 ['card2_103.0', 0.11047070124879924, 3468],
 ['card2_104.0', 0.3298059964726631, 754],
 ['card2_105.0', 0.0, 290],
 ['card2_106.0', 0.0, 262],
 ['card2_107.0', -1, 0],
 ['card2_108.0', 0.011111111111111112, 91],
 ['card2_109.0', 0.0, 83],
 ['card2_110.0', 0.006514657980456026, 309],
 ['card2_111.0', 0.022120190894080925, 45191],
 ['card2_112.0', 0.0, 269],
 ['card2_113.0', 0.026627218934911243, 347],
 ['card2_114.0', 0.01, 101],
 ['card2_115.0', 0.005154639175257732, 195],
 ['card2_116.0', 0.018691588785046728, 109],
 ['card2_117.0', 0.0, 62],
 ['card2_118.0', 0.04519774011299435, 185],
 ['card2_119.0', 0.07441860465116279, 231],
 ['card2_120.0', 0.0, 66],


In [330]:
tabs = pd.DataFrame(tabs, columns=['colname', 'fraud_or', 'col_total_present'])
tabs.head()

,colname,fraud_or,col_total_present
0,ProductCD_C,0.132340,68519
1,ProductCD_H,0.050048,33024
2,ProductCD_R,0.039313,37699
3,ProductCD_S,0.062694,11628
4,ProductCD_W,0.020824,439670


In [335]:
reduced_tabs = tabs[(tabs.fraud_or>0)&(tabs.col_total_present>1000)]

In [338]:
reduced_tabs.colname.tolist()

['ProductCD_C',
 'ProductCD_H',
 'ProductCD_R',
 'ProductCD_S',
 'ProductCD_W',
 'card2_100.0',
 'card2_103.0',
 'card2_111.0',
 'card2_122.0',
 'card2_127.0',
 'card2_130.0',
 'card2_142.0',
 'card2_158.0',
 'card2_161.0',
 'card2_170.0',
 'card2_174.0',
 'card2_177.0',
 'card2_181.0',
 'card2_194.0',
 'card2_199.0',
 'card2_204.0',
 'card2_206.0',
 'card2_215.0',
 'card2_225.0',
 'card2_250.0',
 'card2_264.0',
 'card2_268.0',
 'card2_269.0',
 'card2_276.0',
 'card2_296.0',
 'card2_298.0',
 'card2_302.0',
 'card2_310.0',
 'card2_321.0',
 'card2_327.0',
 'card2_343.0',
 'card2_360.0',
 'card2_361.0',
 'card2_369.0',
 'card2_375.0',
 'card2_383.0',
 'card2_390.0',
 'card2_393.0',
 'card2_399.0',
 'card2_404.0',
 'card2_408.0',
 'card2_417.0',
 'card2_418.0',
 'card2_445.0',
 'card2_452.0',
 'card2_453.0',
 'card2_455.0',
 'card2_470.0',
 'card2_476.0',
 'card2_480.0',
 'card2_481.0',
 'card2_490.0',
 'card2_494.0',
 'card2_500.0',
 'card2_512.0',
 'card2_513.0',
 'card2_514.0',
 'card2_

In [340]:
r_df = df.loc[:,reduced_tabs.colname.tolist()]
del df
gc.collect()

34060

In [342]:
r_df.to_parquet('r_reduced_all_cat.pqt.gz', compression='gzip')

In [348]:
indexes.to_csv('./output/transaction_ids.csv.gz')